# Colab Start Section

Step 1 is to upload a copy of your private key. I am assuming your private key is named id_ed25519. If that isn't the case, you'll have to make manual changes.

In [1]:
# Change these of course to what is your email and your GitHub username
EMAIL = "pauleanderson@gmail.com"
USERNAME = "pauleanderson"
ORG = "anderson-fall-2024"

In [2]:
from google.colab import files
uploaded = files.upload()

Saving id_ed25519 to id_ed25519


In [3]:
%%bash -s "$EMAIL" "$USERNAME"

rm -rf /root/.ssh/*
mkdir /root/.ssh

mv id_ed25519 /root/.ssh/

chmod 600 /root/.ssh/id_ed25519

git config --global user.email "$1"
git config --global user.name "$2"

In [8]:
%%bash -s "$USERNAME" "$ORG"

eval "$(ssh-agent -s)"

ssh-add ~/.ssh/id_ed25519

ssh-keyscan -t rsa github.com >> ~/.ssh/known_hosts

git clone git@github.com:$2/csc-487-$1.git

Agent pid 726


Identity added: /root/.ssh/id_ed25519 (pauleanderson@gmail.com)
# github.com:22 SSH-2.0-babeld-cd94d92b7
Cloning into 'csc-487-instructor'...


In [2]:
%load_ext autoreload
%autoreload 2


from pathlib import Path
home = str(Path.home()) # all other paths are relative to this path. change to something else if this is not the case on your system

import sys
sys.path.insert(0,f"csc-487-{USERNAME}")
sys.path.insert(0,"..")

import py487

In [3]:
# For tips on running notebooks in Google Colab, see
# https://pytorch.org/tutorials/beginner/colab
%matplotlib inline

# Assignment 5 - Implementing your own layers

Paul E. Anderson

## Motivation
We are building our understanding of the different types of layers for a neural network by implementing them from scratch (i.e., pure Python).

## Instructions:
In this assignment you will be coding classes that replicate some of the functionality from:
* https://pytorch.org/docs/stable/_modules/torch/nn/modules/linear.html
* https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html

Your code should stick to 100% pure Python. i.e., no numpy, pandas, and PyTorch. The only exception is these libraries are used to generate input. Your output and code should not use them as anything more than input.

You are free to look at PyTorch source code for inspiration, but I wouldn't chase an exact one-to-one coding port.

## Important implementation notes
Since weights are initialized at random, please use the function I provide in ``py487.nn.get_new_random_weight``. When implementing linear, the outermost loop should be over the nodes. You should also generate the biases after you have generated all other weights.

## Linear

### We need some data for testing purposes

In [2]:
import matplotlib.pyplot as plt
from sklearn import datasets
import numpy as np
np.random.seed(4)
X, t = datasets.make_blobs(n_samples=100, centers=3, n_features=4, center_box=(0, 10))

In [3]:
import torch
torch_layer = torch.nn.Linear(4,10)
W,b = list(torch_layer.parameters())
W = W.detach().numpy().tolist()
b = b.detach().numpy().tolist()
torch_layer(torch.Tensor(X[:4]))

tensor([[-3.0514,  1.9010,  0.7878, -5.5364, -2.1675, -5.3132,  2.0600, -0.0570,
          2.0788,  1.1156],
        [-5.5435,  0.7840, -2.5190, -6.6141, -5.2038, -7.0382,  3.7707, -0.7688,
          1.6381,  4.0393],
        [-2.3709,  2.7171,  0.2324, -2.8502, -0.8866, -5.0129,  3.9327, -2.5185,
          1.1416,  0.2376],
        [-2.9382,  1.9173,  1.7087, -6.3171, -2.0009, -5.5399,  2.0373,  0.0447,
          2.0261,  0.9221]], grad_fn=<AddmmBackward0>)

In [4]:
layer = py487.nn.Linear(4,10)
layer.W = W
layer.b = b
np.array(layer.transform(X[:4]))

array([[-3.05135991,  1.90104992,  0.78777163, -5.5364001 , -2.16748847,
        -5.31318821,  2.0599534 , -0.05696987,  2.07877585,  1.11564758],
       [-5.54351069,  0.78399725, -2.51901847, -6.61408735, -5.20377116,
        -7.03817177,  3.77068254, -0.76881425,  1.63809678,  4.03926829],
       [-2.3708948 ,  2.71705549,  0.23241454, -2.85017997, -0.88664849,
        -5.01288253,  3.93266275, -2.51851691,  1.14156147,  0.23763347],
       [-2.93817657,  1.91729047,  1.70868121, -6.31710108, -2.00086755,
        -5.53994463,  2.03732604,  0.04469909,  2.02605155,  0.92206351]])

In [5]:
layer = py487.nn.Linear(4,10)
np.array(layer.transform(X[:4]))

array([[ 7.78001688, 14.08582519,  5.53625077,  6.82312412,  9.56491452,
        10.14402522,  8.28957839, 14.78423217,  9.78495955, 15.10787483],
       [11.92475096, 21.69189983, 10.61140271, 12.33027784, 14.39880151,
        17.5054005 , 13.76343296, 26.2666809 , 19.96634931, 25.75031696],
       [ 5.41777825, 13.35565848,  4.72635997,  8.73640488, 10.81105103,
         8.95051464,  5.01725939, 14.4863225 , 11.48882949, 14.68151436],
       [ 7.99175032, 14.34280588,  5.60751452,  7.18037327,  9.84307807,
         9.76382783,  8.02227742, 14.85178769,  9.06337085, 15.26008802]])

## Conv2d

For this standardized assignment, you will implement a limited version of PyTorch's Conv2d. Your implementation
should support any number of inputs, any number of input channels, and any size image. Your implementation can be limited in the following sense:
* You can assume a kernel size of 3.
* You can assume there will be no bias.
* You can assume that padding will always equal 1.

Below you can find an example execution of my solution to this problem. Please see the test cases and use those to guide you to make incremental progress on this one. 

We first need a dataset of course. We will create one that has 20 images with 16 channels that are sized 32 x 32.

In [19]:
input = torch.randn(20, 16, 32, 32)

In [21]:
channel_avg = np.mean(input.detach().numpy(), (0, 2, 3))
channel_avg

array([ 0.00247935,  0.00623732,  0.00315957, -0.00350458, -0.00423463,
       -0.00123584,  0.00467167, -0.01315151, -0.00987491,  0.00572499,
       -0.00198541,  0.00123873, -0.00317038, -0.00281763,  0.00249892,
       -0.01791426], dtype=float32)

In [22]:
import torch.nn as nn
# With square kernels and equal stride
m = nn.Conv2d(16, 2, 3, padding=1,bias=False)
output = m(input)
print("The output shape is",output.shape)

The output shape is torch.Size([20, 2, 32, 32])


In [23]:
output = m(input) 
print("An example output for one image is:\n",output[:1])

An example output for one image is:
 tensor([[[[ 0.3559, -0.4042,  0.0676,  ..., -0.5221,  0.0112, -0.3082],
          [ 0.0387, -0.0678, -0.5145,  ...,  0.1490, -0.1955, -0.2780],
          [ 0.6575,  0.1259, -0.3579,  ..., -1.0116,  0.2763,  1.1288],
          ...,
          [-1.0054,  0.5265, -0.3163,  ...,  0.5941, -0.5093,  0.1143],
          [ 0.2613,  0.0561,  1.3304,  ...,  0.0557, -0.8539, -0.6020],
          [ 0.3314,  1.0446,  0.4888,  ...,  0.3743,  0.1206,  0.6735]],

         [[-0.2228, -0.2913,  0.1243,  ..., -0.1958,  0.9448,  0.2423],
          [ 0.4050, -0.8187,  0.5489,  ...,  0.6726, -0.1190, -0.1260],
          [-0.4432,  0.0939,  0.4535,  ..., -0.1388,  0.1142, -0.7076],
          ...,
          [ 0.1688, -0.8324,  1.0597,  ..., -0.3439,  0.0186,  0.9987],
          [ 0.5498, -0.0174, -0.3927,  ..., -0.5744,  0.1671, -0.1172],
          [ 0.4285, -0.3679, -0.2670,  ..., -1.5167, -0.5148, -0.5526]]]],
       grad_fn=<SliceBackward0>)


In [24]:
print("Our weight matrix that contains our kernels has the following shape:\n",m.weight.shape)

Our weight matrix that contains our kernels has the following shape:
 torch.Size([2, 16, 3, 3])


**Our implementation now:**

In [25]:
layer = py487.nn.Conv2d(16, 2, 3)
print("First, your shape should match PyTorch:\n",np.array(layer.W).shape)

First, your shape should match PyTorch:
 (2, 16, 3, 3)


If that is true, you can use PyTorch's weight matrix

In [26]:
W = m.weight.detach().numpy().tolist()
layer.W = W

In [27]:
print("Second, we can check whether our output is the correct shape:\n",np.array(layer.transform_dryrun(input)).shape)

Second, we can check whether our output is the correct shape:
 (20, 2, 32, 32)


In [28]:
print("Here is an example dryrun output (zeros are what you should see):\n",np.array(layer.transform_dryrun(input))[0])

Here is an example dryrun output (zeros are what you should see):
 [[[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]]


In [29]:
print("Now we can test our convolution:\n",np.array(layer.transform(input))[0])

Now we can test our convolution:
 [[[ 0.35593852 -0.40422426  0.06760901 ... -0.52211441  0.01118538
   -0.30821643]
  [ 0.03867475 -0.06777543 -0.51450794 ...  0.14903607 -0.19553183
   -0.27804863]
  [ 0.65753443  0.12586502 -0.35789976 ... -1.01161504  0.27625826
    1.12881637]
  ...
  [-1.00535865  0.52653284 -0.31631291 ...  0.59405306 -0.50928068
    0.11427801]
  [ 0.26130189  0.056141    1.33041357 ...  0.05569568 -0.8539353
   -0.60195998]
  [ 0.33139465  1.04463859  0.48882595 ...  0.37427971  0.12061934
    0.67353672]]

 [[-0.22280776 -0.29134453  0.12429806 ... -0.19581417  0.9448387
    0.24225088]
  [ 0.40500557 -0.8186519   0.54891695 ...  0.67256474 -0.11899789
   -0.12602717]
  [-0.44321089  0.09394567  0.453458   ... -0.13879769  0.11422061
   -0.70762143]
  ...
  [ 0.16876471 -0.83235505  1.05969282 ... -0.34392846  0.01857664
    0.99870993]
  [ 0.54983704 -0.01737289 -0.39265563 ... -0.57444189  0.16713793
   -0.11715195]
  [ 0.42845175 -0.36791493 -0.26703198 ..

In [28]:
# End of notebook